In [1]:
using StatisticalRethinking, Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

352×2 Array{Float64,2}:
 1.0  47.8256
 1.0  36.4858
 1.0  31.8648
 1.0  53.0419
 1.0  41.2769
 1.0  62.9926
 1.0  38.2435
 1.0  55.48  
 1.0  34.8699
 1.0  54.4877
 ⋮           
 1.0  47.8823
 1.0  39.4058
 1.0  41.0501
 1.0  40.8233
 1.0  47.0318
 1.0  34.2462
 1.0  52.1631
 1.0  54.0625
 1.0  52.5316

Model Specification

In [3]:
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"
NaN


Initial Values

In [4]:
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

3-element Array{Dict{Symbol,Any},1}:
 Dict(:beta=>[290.944, 7.32004],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>18.4792) 
 Dict(:beta=>[89.5338, 11.8022],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>20.9178) 
 Dict(:beta=>[114.129, -19.4873],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>23.1864)

Tuning Parameters

In [5]:
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

0.1

Define sampling scheme

In [6]:
scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"
NaN


MCMC Simulation

In [7]:
sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:14:60 of 0:15:00 remaining]
Chain 1:  10% [0:00:60 of 0:01:06 remaining]
Chain 1:  20% [0:00:58 of 0:01:12 remaining]
Chain 1:  30% [0:00:52 of 0:01:14 remaining]
Chain 1:  40% [0:00:45 of 0:01:16 remaining]
Chain 1:  50% [0:00:37 of 0:01:14 remaining]
Chain 1:  60% [0:00:30 of 0:01:14 remaining]
Chain 1:  70% [0:00:22 of 0:01:15 remaining]
Chain 1:  80% [0:00:15 of 0:01:14 remaining]
Chain 1:  90% [0:00:07 of 0:01:14 remaining]
Chain 1: 100% [0:00:00 of 0:01:14 remaining]

Chain 2:   0% [0:00:07 of 0:00:07 remaining]
Chain 2:  10% [0:00:53 of 0:00:59 remaining]
Chain 2:  20% [0:00:56 of 0:01:09 remaining]
Chain 2:  30% [0:00:50 of 0:01:12 remaining]
Chain 2:  40% [0:00:44 of 0:01:14 remaining]
Chain 2:  50% [0:00:38 of 0:01:15 remaining]
Chain 2:  60% [0:00:30 of 0:01:14 remaining]
Chain 2:  70% [0:00:22 of 0:01:14 remaining]
Chain 2:  80% [0:00:15 of 0:01:15 remaining]
Chain 2:  90% [0:00:08 of 0:01:16 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[30.5645 112.241 0.939701; 23.9761 113.009 0.925999; … ; 26.4892 114.124 0.886743; 25.644 113.782 0.895372]

[26.9448 114.583 0.899226; 27.3186 114.584 0.885683; … ; 28.8019 110.663 0.975316; 26.7791 111.214 0.966822]

[25.038 110.215 0.976471; 26.1712 110.215 0.976471; … ; 25.1718 115.275 0.874487; 26.5515 115.275 0.874487]

Show draws summary

In [8]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean        SD       Naive SE       MCSE        ESS   
     s2  26.1749563 1.98754677 0.0120958245 0.0189454250 9000.0000
beta[1] 113.9435719 1.90700923 0.0116056886 0.0357703011 2842.2377
beta[2]   0.9036673 0.04195967 0.0002553584 0.0007788297 2902.5462

Quantiles:
            2.5%         25.0%        50.0%       75.0%        97.5%   
     s2  22.57283163  24.79721389  26.06923887  27.4445829  30.31433738
beta[1] 110.23166347 112.64725299 113.94424085 115.2323768 117.67645899
beta[2]   0.82150273   0.87550054   0.90363053   0.9322818   0.98556096



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*